# Intermediate Project - SAAS Career Exploration - Part 1

For this project, you will be using the tools that you have learned so far on a real data science problem. You will be cleaning and analyzing a dataset to answer a research question, something that you will be doing for your entire career if you continue down this path.

This project will be done **in groups**. Modern research is collaborative, so get used to it! If you are having trouble finding a partner, please contact your committee director or post on the Career-Exploration Slack channel.

Parts that you need to complete <span style="color:blue">will be written in blue</span> and have `#TODO` next to them.

The final product for this project will be a statistical model that answers a question posed about the data, in addition to a short description about how your model works and its limitations.

## 1. The Dataset

### 2016 US Election from Kaggle

https://www.kaggle.com/benhamner/2016-us-election/data

This dataset gives results from the 2016 Democratic primaries for the US presidential election. The results themselves are stored in the ``primaries`` dataframe, while information about each county and state is stored in the ``general`` dataframe.


In [73]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd

general = pd.read_csv("2016-us-election/county_facts.csv")
column_dict_df = pd.read_csv("2016-us-election/county_facts_dictionary.csv")
primaries = pd.read_csv("2016-us-election/primary_results.csv")

### 1.1. Cleaning general

First, we have to clean the data to make it a little bit easier for us to use. What's wrong with it now? <span style="color:blue">Well, let's start off by examining the `general` dataframe using the `.head()` command.</span>

In [75]:
# TODO: Your code here
general.head()

,fips,area_name,state_abbreviation,PST045214,PST040210,PST120214,POP010210,AGE135214,AGE295214,AGE775214,...,SBO415207,SBO015207,MAN450207,WTN220207,RTN130207,RTN131207,AFN120207,BPS030214,LND110210,POP060210
0,0,United States,NaN,318857056,308758105,3.3,308745538,6.2,23.1,14.5,...,8.3,28.8,5319456312,4174286516,3917663456,12990,613795732,1046363,3531905.43,87.4
1,1000,Alabama,NaN,4849377,4780127,1.4,4779736,6.1,22.8,15.3,...,1.2,28.1,112858843,52252752,57344851,12364,6426342,13369,50645.33,94.4
2,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,13.8,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
3,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,18.7,...,1.3,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6
4,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,16.5,...,0.0,27.0,0,0,188337,6334,0,8,884.88,31.0


Oh no! What are those column names? Why do we have states and counties in the same table? Why don't the states have abbreviations? Dealing with problems like these is called *data cleaning*, and is frequently one of the most important and most time-intensive parts of data science. Lucky for you, I've done the data cleaning already! <span style="color:blue">Skim through the code below just to get a general idea of what's going on, but please don't worry about every last detail of what it does.</span>

In [76]:
# Turn the county_facts_dictionary.csv file into a dictionary
column_dict = column_dict_df.set_index("column_name").to_dict()['description']

# Use that dictionary to rename the columns of general
general.columns = general.columns.to_series().map(lambda x: column_dict.get(x,x))

# Extract the rows corresponding to states from general (note that these are the rows with NaN in the 
# state_abbreviation column, minus the first row which is the whole US)
states = general[general['state_abbreviation'].isnull()][1:].reset_index(drop=True)

# Attach the state abbreviations to the states dataframe
states["state_abbreviation"] = general["state_abbreviation"].unique()[1:]

# Extract the rows corresponding to counties from general
counties = general[~general['state_abbreviation'].isnull()].reset_index(drop=True)

In [77]:
states.head()

,fips,area_name,state_abbreviation,"Population, 2014 estimate","Population, 2010 (April 1) estimates base","Population, percent change - April 1, 2010 to July 1, 2014","Population, 2010","Persons under 5 years, percent, 2014","Persons under 18 years, percent, 2014","Persons 65 years and over, percent, 2014",...,"Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Manufacturers shipments, 2007 ($1,000)","Merchant wholesaler sales, 2007 ($1,000)","Retail sales, 2007 ($1,000)","Retail sales per capita, 2007","Accommodation and food services sales, 2007 ($1,000)","Building permits, 2014","Land area in square miles, 2010","Population per square mile, 2010"
0,1000,Alabama,AL,4849377,4780127,1.4,4779736,6.1,22.8,15.3,...,1.2,28.1,112858843,52252752,57344851,12364,6426342,13369,50645.33,94.4
1,2000,Alaska,AK,736732,710249,3.7,710231,7.4,25.3,9.4,...,0.0,25.9,8204030,4563605,9303387,13635,1851293,1518,570640.95,1.2
2,4000,Arizona,AZ,6731484,6392310,5.3,6392017,6.4,24.1,15.9,...,10.7,28.1,57977827,57573459,86758801,13637,13268514,26997,113594.08,56.3
3,5000,Arkansas,AR,2966369,2915958,1.7,2915918,6.5,23.8,15.7,...,2.3,24.5,60735582,29659789,32974282,11602,3559795,7666,52035.48,56.0
4,6000,California,CA,38802500,37254503,4.2,37253956,6.5,23.6,12.9,...,16.5,30.3,491372092,598456486,455032270,12561,80852787,83645,155779.22,239.1


In [78]:
counties.head()

,fips,area_name,state_abbreviation,"Population, 2014 estimate","Population, 2010 (April 1) estimates base","Population, percent change - April 1, 2010 to July 1, 2014","Population, 2010","Persons under 5 years, percent, 2014","Persons under 18 years, percent, 2014","Persons 65 years and over, percent, 2014",...,"Hispanic-owned firms, percent, 2007","Women-owned firms, percent, 2007","Manufacturers shipments, 2007 ($1,000)","Merchant wholesaler sales, 2007 ($1,000)","Retail sales, 2007 ($1,000)","Retail sales per capita, 2007","Accommodation and food services sales, 2007 ($1,000)","Building permits, 2014","Land area in square miles, 2010","Population per square mile, 2010"
0,1001,Autauga County,AL,55395,54571,1.5,54571,6.0,25.2,13.8,...,0.7,31.7,0,0,598175,12003,88157,131,594.44,91.8
1,1003,Baldwin County,AL,200111,182265,9.8,182265,5.6,22.2,18.7,...,1.3,27.3,1410273,0,2966489,17166,436955,1384,1589.78,114.6
2,1005,Barbour County,AL,26887,27457,-2.1,27457,5.7,21.2,16.5,...,0.0,27.0,0,0,188337,6334,0,8,884.88,31.0
3,1007,Bibb County,AL,22506,22919,-1.8,22915,5.3,21.0,14.8,...,0.0,0.0,0,0,124707,5804,10757,19,622.58,36.8
4,1009,Blount County,AL,57719,57322,0.7,57322,6.1,23.6,17.0,...,0.0,23.2,341544,0,319700,5622,20941,3,644.78,88.9


### 1.2. Cleaning primaries

Next, let's look at the `primaries` dataframe.

In [140]:
primaries.head()

,state,state_abbreviation,county,fips,party,candidate,votes,fraction_votes
0,Alabama,AL,Autauga,1001.0,Democrat,Bernie Sanders,544,0.182
1,Alabama,AL,Autauga,1001.0,Democrat,Hillary Clinton,2387,0.800
2,Alabama,AL,Baldwin,1003.0,Democrat,Bernie Sanders,2694,0.329
3,Alabama,AL,Baldwin,1003.0,Democrat,Hillary Clinton,5290,0.647
4,Alabama,AL,Barbour,1005.0,Democrat,Bernie Sanders,222,0.078


For future parts of this project, it would be much easier for us if we only had one row per county which had the votes for all the candidates.<span style="color:blue"> Do this using a pivot table where the index is the state *and the county* (you can pass a list of multiple column names into the `index` argument), the columns are the candidates, and the values are the fraction of votes that each candidate received. </span> Hint: You can use the same function from lecture, just replace the arguments appropriately.

In [127]:
# TODO: Create the pivot table as described above.
pivot_table = pd.pivot_table(primaries,index=["state","county"],columns="candidate",values='fraction_votes')

In [128]:
# TODO: Run this cell after filling out the above to make the indices into columns.
pivot_table.reset_index(inplace = True)
pivot_table.head()

candidate,state,county,No Preference,Uncommitted,Ben Carson,Bernie Sanders,Carly Fiorina,Chris Christie,Donald Trump,Hillary Clinton,Jeb Bush,John Kasich,Marco Rubio,Martin O'Malley,Mike Huckabee,Rand Paul,Rick Santorum,Ted Cruz
0,Alabama,Autauga,NaN,NaN,0.146,0.182,NaN,NaN,0.445,0.800,NaN,0.035,0.148,NaN,NaN,NaN,NaN,0.205
1,Alabama,Baldwin,NaN,NaN,0.084,0.329,NaN,NaN,0.469,0.647,NaN,0.059,0.193,NaN,NaN,NaN,NaN,0.170
2,Alabama,Barbour,NaN,NaN,0.122,0.078,NaN,NaN,0.501,0.906,NaN,0.036,0.146,NaN,NaN,NaN,NaN,0.179
3,Alabama,Bibb,NaN,NaN,0.099,0.197,NaN,NaN,0.494,0.755,NaN,0.021,0.112,NaN,NaN,NaN,NaN,0.255
4,Alabama,Blount,NaN,NaN,0.100,0.386,NaN,NaN,0.487,0.551,NaN,0.022,0.122,NaN,NaN,NaN,NaN,0.244


### 1.3. Unique counties

Why do we need to index by both state and county? Well, it turns out that there are tons of counties with duplicate names! We wouldn't want to accidentally combine all of the counties named "Calhoun County" over all the states with that county. (There's a "Calhoun County" in eleven states!!) <span style="color:blue"> Let's try to figure out how many unique county names there are; use the `.unique()` function on the `county` column to find the number of unique column names, and compare that with the number of rows in the pivot table. </span>

In [130]:
# TODO: Find the number of unique county names.
uniques = pivot_table['county'].unique() # YOUR CODE HERE
len(uniques)

2633

In [131]:
# TODO: Find the number of rows in pivot_table to see how many different counties there actually are.
len(pivot_table)

4216

### 1.4. Lake County

More than a third of the counties don't have a unique name! <span style="color:blue"> The county I'm from is called "Lake County" - how many different states could I be from?</span>

In [134]:
# TODO: Find the number of states with a county named "Lake" (or the county you're from if you'd prefer).
primaries.loc[primaries['county']=='Lake','state'].unique()

array(['California', 'Colorado', 'Florida', 'Illinois', 'Indiana',
       'Michigan', 'Montana', 'Ohio', 'Oregon', 'South Dakota',
       'Tennessee'], dtype=object)

### 1.5. Challenge problem! (Optional)

CHALLENGE PROBLEM: Find the county name that is duplicated the most often. If you can, try doing this without any loops!

In [142]:
# TODO: Your code here
uniques = pivot_table['county'].unique()
max_county = uniques[0]
max_count = 0
for county in uniques:
    cur_count = len(primaries.loc[primaries['county']==county,'state'].unique())
    if cur_count > max_count:
        max_county = county
        max_count = cur_count
print(max_county,max_count)

In [150]:
counts = pivot_table.groupby('county').count().iloc[:,0]
max_count = counts.max()
counts[counts == max_count]

county
Washington    30
Name: state, dtype: int64

## 2. Submission

**To submit, first save this file as a pdf by going to the top left and clicking File -> Download as -> PDF via LaTex (.pdf), then fill out this form!**

https://docs.google.com/forms/d/e/1FAIpQLSfC959ud0v9C9vZBEM2U41ryAQA5DsgU4d56_BnSVCfYEsTZw/viewform?fbclid=IwAR0s64G8p3U2NNFgzNHTV6vuAxCLd1redLc6SEgW77gCNfx39p8VORhPkt4